![Gambar](https://neurohive.io/wp-content/uploads/2018/11/vgg16-1-e1542731207177.png)

![](https://machinelearningknowledge.ai/ezoimgfmt/953894.smushcdn.com/2611031/wp-content/uploads/2020/08/Keras-Implementation-of-VGG16-Architecture-2.jpg?lossy=0&strip=1&webp=1&ezimgfmt=ng:webp/ngcb1)


Spesifikasi Perangkat :
- AMD Ryzen 5600X 6C12T
- Nvidia RTX 3060 12GB
- RAM 16 GB

In [1]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models

In [2]:
def VGGConvLayer(filters):
    return layers.Conv2D(filters=filters,kernel_size= (3,3), padding='same', activation='relu')



def VGGMaxPool():
    return layers.MaxPool2D((2,2), strides=(2,2))

In [3]:
def VGG16(num_classes:int):
    model = models.Sequential()
    for _ in range(2):
        model.add(layers.Conv2D(filters=64,kernel_size= (3,3), padding='same', activation='relu', input_shape=(224,224,3)))
    model.add(VGGMaxPool())
    for _ in range(2):
        model.add(VGGConvLayer(128))
    model.add(VGGMaxPool())
    for _ in range(3):
        model.add(VGGConvLayer(256))
    model.add(VGGMaxPool())
    for _ in range(3):
        model.add(VGGConvLayer(512))
    model.add(VGGMaxPool())
    for _ in range(3):
        model.add(VGGConvLayer(512))
    model.add(VGGMaxPool())
    model.add(layers.Flatten())
    for _ in range(2):
        model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model

In [4]:
model = VGG16(num_classes=16)
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 128)      0         
 2D)                                                    

Dataset

In [5]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory('Dataset/flowers', validation_split=0.2, subset='training', seed=1337, image_size=(224,224), batch_size=4)
val_ds = tf.keras.preprocessing.image_dataset_from_directory('Dataset/flowers', validation_split=0.2, subset='validation', seed=1337, image_size=(224,224), batch_size=4)

Found 15740 files belonging to 16 classes.
Using 12592 files for training.
Found 15740 files belonging to 16 classes.
Using 3148 files for validation.


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
import datetime
import os





timestr = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
name = 'vgg-16-'+timestr

checkpoint_path = "checkpoints/"+name+"/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
os.system('mkdir {}'.format(checkpoint_dir))

# save model after each epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1
)
tensorboard_callback = TensorBoard(
    log_dir='tensorboard_logs/'+name,
    histogram_freq=1
)

model.fit(
    train_ds,
    epochs=20,
    verbose=1,
    validation_data= val_ds,
    batch_size=4,
    callbacks=[cp_callback, tensorboard_callback]
)